One reason that the solver is currently pretty slow is that we waste a lot of time generating solutions based on partial parses that could never be part of the final answer. We need some way to push output constraints from the top down based on the known answer pattern (and the fact that the answer has to actually be a word or phrase). But I'm not totally sure how to handle that information yet. 

First of all, what constraints are available: 

Clue: answer length and possible letter pattern, must be a word (or phrase)

Clue -> (Wordplay, Definition): 
Propagates the same top-level constraints to the Wordplay

Wordplay -> (Wordplay, Wordplay):

Wait a minute, can I just work backwards from the solution? That is, given a parse structure, can I determine if a given word can be explained by that parse? That's still tricky, because the parse structure doesn't really tell us enough to determine that easily. For example, if our parse contains an (InsertAB, Wordplay, Wordplay), we can't uniquely identify the two separate pieces that were combined to give some known result. So we'd still end up iterating over all possible insertions, just in reverse. 

Ok, back to the constraint info: 

Assume the given length is N and the given pattern is `[p1, ..., pN]`

Clue -> (Wordplay, Definition): 
Propagates the same top-level constraints to the Wordplay

Wordplay -> (Wordplay, Wordplay):
Left must be length `[1, N - 1]`, right must be length `[1, N - 1]`. 
Left must be a valid word prefix and must match a prefix of the pattern, right must
be a valid word suffix and must match a suffix of the pattern

Wordplay (length `[N, M]`) -> (InsertAB, Wordplay, Wordplay):
Each argument wordplay must have length `[N - 1, M - 1]`

It seems like what I need to do is give each Arc a Context expressing these constraints. But who actually owns that context? It doesn't really make sense during the parsing process, since an Arc doesn't actually have a unique context (e.g. a given Token could be a member of many different wordplays, each of which will have its own context). 

However, it seems like contexts would work fine if parsing is separate from solving, since I can do something like:

```
function solve(arc, rule, context)
  args = [solve(arg, arg.rule, propagate(rule, context)) for arg in arc.args]
  combine(rule, args)
end
```



Next issue: how do I make the general parser cryptic-worthy? I can easily make the parser agnostic to the exact type of rule in the grammar, but there are a few nagging issues. 


1. I shouldn't assume that the rules are actually Pairs. After all, I may end up using something like FunctionPointers to avoid dynamic dispatch, so my Rule type might just be some generic struct. A rule just needs to provide access to its head and arguments. 

2. How do I deal with leaf parses, particularly phrases? Here are some options: 

    A. Every word just produces a Token() in the parser, and the parser is agnostic to the actual word. This is easy to implement, but I'm concerned that we could end up with a lot of complete parses that end up relying on nonsense assignments like (AnagramIndicator -> "a"). We can try to catch those during the solving process, but it seems like it's harder to filter them out of the chart than to avoid having them enter in the first place. 
    B. Add an inactive arc of the form (WhateverIndicator -> ()) for each possible assignment of each word. This feels more complicated than the previous suggestion, but I don't think it makes parsing actually any harder (it just removes the unlabeled Token() symbol). I'm most concerned about how to handle phrases in this way. Perhaps we don't need to be totally generic in handling phrases? Perhaps we could initialize the list with: 1) every word and each of its possible assignments, 2) every adjacent two-word phrase and the indicators they could represent, 3) every initial and final phrase of up to N-1 words as a possible definition. That only adds a number of extra rules which is linear in the length of the clue. Even adding every possible phrase is only quadratic in the length of the clue, which doesn't seem so bad. 
   
I think option B is more attractive, particularly because I want to be able to include probabilistic information into the parse results (eventually), and I suspect that the assignments of words to symbols will be the primary source of that probability information. 
    
    
This also ties into how I decide to handle filler words. 



Ok, another question: if I end up using contexts and solving after parsing, how do I avoid re-solving a given arc for every possible parent that contains it? In python, I did this by caching the solve results for each arc, but that caching now has to also depend on the context. 

It seems like each Arc needs to contain a cache mapping context -> vector of string outputs. When solving an arc in a given context, we would need to: 

1. Check if the given context is in the cache. If so, return the cached result
2. For each context in the cache, if the input context is a subset of that context, then just filter the cached result by the new (more restrictive) context, add that to the cache, and return it. 
3. Otherwise, actually apply the given rule and cache its output. 


Let's explore this idea a little further. Let's assume that the grammar consists of generic production rules and a set of word and phrase assignments which will be used to generate the initial chart or agenda (depending on the parsing strategy). 

What can a word be? 

* Literal
* AnagramIndicator
* SubstringIndicator
* ReverseIndicator
* InsertIndicator
* Definition
* Nothing

(with a few variants of Substring and Insert that I won't get into here). 

A phrase can be: 

* *Indicator
* Definition

(so, all the things a word can be except a Literal)

Define the operations in terms of those components: 

* Clue => (Wordplay, Definition)
* Clue => (Definition, Wordplay)
* Wordplay => (Wordplay, Wordplay) by concatenation
* Wordplay => (ReverseIndicator, Wordplay) reversal
* Wordplay => (AnagramIndicator, Literal) anagram
* Wordplay => (InsertIndicator, Wordplay, Wordplay) insert
* Wordplay => (Literal) identity
* Wordplay => (Wordplay, Nothing) eliminate the Nothing entry (note: we could have done Wordplay => (Nothing) directly, but that would violate the assumption that each wordplay is at least 1 character)

How do we handle the bookkeeping associated with multiple solutions from a single arc? Each arc, when solved in a given context, will have a vector of possible inputs for each of its argument. Each combination of input arguments can, furthermore, produce multiple outputs. So for each output, we need to record either the arguments that were used to produce it or some indicator of which argument was used, like the index into the vector of values of each input. Recording the indices might be more efficient, but it requires that we actually store that vector of possibilities for each input, which we might not want to do. Just storing the strings (which should just require a pointer for each) is probably the way to go. 


Ok, here's another idea: 

Each arc carries a context, a list of inputs, and a single output. When applying the fundamental rule, we check whether the newly added argument satisfies the context into which it is being added. When generating hypotheses, we always work top-down and propagate the context information. We only ever generate hypotheses once for a given (symbol, start, context) triplet. 

There's a risk this could backfire, as we will probably end up storing many copies of the same arc with the same output, reached through different contexts. In the idea presented above (parse then solve), this is sort of covered because we can examine the cache to see if there is a superset context we can filter to avoid re-solving the arc in a new context. On the other hand, the whole superset thing might prove difficult to implement in practice, since it's going to be hard to combine with letter patterns. 

Of course, this is actually orthogonal to the caching idea: it would still be possible to have an arc contain a single context (and hypothesize new arcs for each context) but store multiple outputs in a single arc. 

This still leaves open the fundamental question: parse then solve or solve while parsing? Is it better to avoid solves which can't contribute to complete parses, or does the solving process provide enough information to eliminate entire parses that can't contribute actual solutions? 

Another thing to think about: we don't have to blindly split up the context. What we actually need to be able to do is to take a context for a given arc and then find a sub-context for that arc with 0 or more of its arguments fixed. So, for example, given a concatenation rule with the initial context (word, 7-7), we would first hypothesize a wordplay with context (word, 7-7) for the first argument. But later, when we are hypothesizing a wordplay for the second argument, that arc will have an actual value attached to its first argument, say, "foo", so we can pass the much more accurate context (word, 4-4) when hypothesizing the second argument. 

That actually means we want a new context for each possible value of each input, which fits nicely with the idea of generating a new arc for each output. 

Okay, it seems like I have some concrete steps I can take. First, a slightly more manual handling of phrases (rather than just allowing tokens to combine into more tokens) seems prudent. For now, I can just remove phrases entirely. Second, I think the context idea is important, even if it might generate even more arcs in some cases. This also creates the opportunity to generalize what I did in the python solver when I couldn't find a good parse, which was to just do a brute-force search over words matching the given pattern. I can just add a low-probability rule to the grammar which is something like: Clue -> (Definition, Unknown) which somehow searches just over matches of the definition and the context. 

I still really want to avoid wasting time generating the same hypotheses for slightly different contexts. One easy improvement is, when I'm about to add a hypothesis to the agenda, do the following: 

1. Check if the (symbol, start, context) triplet was already used
2. If yes, then quit
3. If not, then check if that (symbol, start) pair was already used with some other context
4. If yes, and if that context is a superset of the current context, then quite
5. If not, then actually add the hypothesis to the agenda. 

That should certainly reduce the number of duplicates, but I'm not sure it's enough. Perhaps there's a more fundamental point: once an arc is inactive, its context is no longer relevant. After all, that context would only serve to restrict the set of outputs the arc could have, but the arc already *has* an output, so the context is done. The only way into the chart is through the agenda, so we just need to ensure that we maintain a set of every inactive arc which has ever been added to the agenda, where that set membership is checked without regard for the arc's context. 